# Pre-wrangling some data
- Remove 'commerce' from some schools
- Wrangling for datatable (schools, cca, special progs)
- achievement

In [1]:
import json

In [2]:
data = json.load(open("../data/all-schools-info.json"))

In [3]:
# remove commerce
def rmCommerce(data):
    for code, school in data.items():
        if 'L1R5History' in school.keys():
            score = school['L1R5History']['2018']
            if 'Commerce' in score.keys() and score['Commerce'][0] == "-":
                del score['Commerce']
    return data

In [4]:
data = rmCommerce(data)

In [5]:
# wrangling for datatable
for code, school in data.items():
    subject = [{'Type': 'Subject', 'Category': 'N.A.', 'Offering': subject} for subject in school['SubjectOffered']]
    cca = [{'Type': 'CCA', 'Category': type, 'Offering': cca} for type, ccas in school['Cca'].items() for cca in ccas]
    progs = []
    for progType, prog in school['SpecialProgrammes'].items():
        if type(prog) == list and prog[0] != 'Not Available':
            for item in prog:
                progs.append({'Type': 'Special Programmes', 'Category': progType, 'Offering': item})
        if type(prog) == dict:
            for key, value in prog.items():
                progs.append({'Type': 'Special Programmes', 'Category': progType, 'Offering': value})
        if type(prog) == str:
            progs.append({'Type': 'Special Programmes', 'Category': progType, 'Offering': prog})
    school['SubjectOffered'] = subject
    school['Cca'] = cca
    school['SpecialProgrammes'] = progs

In [6]:
data['0701']['SubjectOffered']

[{'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 BIOLOGY'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 CHEMISTRY'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 CHINESE LANGUAGE'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 ECONOMICS'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 GENERAL PAPER'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 GEOGRAPHY'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 HISTORY'},
 {'Type': 'Subject',
  'Category': 'N.A.',
  'Offering': 'H1 LITERATURE IN ENGLISH'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 MALAY LANGUAGE'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 MATHEMATICS'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 PHYSICS'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 PROJECT WORK'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': 'H1 TAMIL LANGUAGE'},
 {'Type': 'Subject', 'Category': 'N.A.', 'Offering': '

In [7]:
# what types of achievement are there
keys = set()
for code, school in data.items():
    for level in school['AchievementHistory'].keys():
        for year in range(2013, 2019):
            for key in school['AchievementHistory'][level][str(year)].keys():
                keys.add(key)
print(keys)

{'SYF Arts Presentation', 'Sustained Achievement Award', 'School Distinction Awards', 'Development Award', 'School Excellence Awards', 'Best Unit Award', 'Achievement Award', 'Special Awards', 'Outstanding Development Award', 'Best Practice Awards', 'Sports & Games Competition'}


In [8]:
for code, school in data.items():
    awards = []
    for level in school['AchievementHistory'].keys():
        for year in range(2014, 2019):
            for key in school['AchievementHistory'][level][str(year)].keys():
                if key not in ['SYF Arts Presentation', 'Sports & Games Competition', 'Best Unit Award']:
                    awards.append({'Year': year, 'Category': 'Others', 'Award': key, 'CCA': 'N.A.'})
                elif key == 'Sports & Games Competition':
                    columns = school['AchievementHistory'][level][str(year)][key]
                    for col in columns[3:]:
                        for i in range(1, len(col)):
                            if col[i] != '-':
                                awards.append({'Year': year, 'Category': "Sports", 'Award': columns[0][i] + ' ' + col[i], 'CCA': col[0] + ' (' + columns[1][i] + ' ' + columns[2][i] + ')'})
                else:
                    for org, award in school['AchievementHistory'][level][str(year)][key].items():
                        if key == "Best Unit Award":
                            awards.append({'Year': year, 'Category': "Best Unit", 'Award': award, 'CCA': org})
                        if key == "SYF Arts Presentation":
                            awards.append({'Year': year, 'Category': "SYF Arts", 'Award': award, 'CCA': org})
    school['awards'] = awards

In [9]:
import pandas as pd

In [10]:
test = pd.DataFrame(data['3047']['awards']).groupby(['Year', 'Category'])['Award'].value_counts().reset_index(name='count').groupby(['Year', 'Category']).agg(list).reset_index().to_json(orient='records')

In [11]:
test = json.loads(test)

In [12]:
for record in test:
    record["Total"] = sum(record["count"])
    details = []
    for i in range(0, len(record["Award"])):
        details.append({'Award': record["Award"][i], 'Count': record["count"][i]})
    record["Details"] = details

In [13]:
# those not working are because no achievement history
for code, school in data.items():
    try:
        award_map = pd.DataFrame(school['awards'])
        award_map = award_map.groupby(['Year', 'Category'])['Award'].value_counts().reset_index(name='count').groupby(['Year', 'Category']).agg(list).reset_index().to_json(orient='records')
        award_map = json.loads(award_map)
        for record in award_map:
            record["Total"] = sum(record["count"])
            details = []
            for i in range(0, len(record["Award"])):
                details.append({'Award': record["Award"][i], 'Count': record["count"][i]})
            record["Details"] = details
        school["awards_map"] = award_map
    except:
        print(code)

1271
1272
3077


In [14]:
for code, school in data.items():
    if 'awards_map' not in school.keys():
        school['awards_map'] = []

In [15]:
json.dump(data, open("../data/school-info-wrangled.json", "w"))